## 🛎️ ⭐ Michelin Restaurant Star Prediction

Given *data about Michelin starred restaurants*, let's try to predict the **number of stars** of a given restaurant.

We will use a logistic regression model to make our predictions. 

Data source: https://www.kaggle.com/datasets/jackywang529/michelin-restaurants

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [2]:
one_star_df = pd.read_csv("archive/one-star-michelin-restaurants.csv")
two_star_df = pd.read_csv("archive/two-stars-michelin-restaurants.csv")
three_star_df = pd.read_csv("archive/three-stars-michelin-restaurants.csv")

In [3]:
one_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...
1,Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Esszimmer,2019,47.806850,13.03409,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
3,Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
4,Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [4]:
two_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,SENNS.Restaurant,2019,47.83636,13.06389,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
1,Ikarus,2019,47.79536,13.00695,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Mraz & Sohn,2019,48.23129,16.37637,Wien,Austria,1200,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
3,Konstantin Filippou,2019,48.21056,16.37996,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
4,Silvio Nickol Gourmet Restaurant,2019,48.20558,16.37693,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [5]:
three_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Amador,2019,48.25406,16.35915,Wien,Austria,1190,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
1,Manresa,2019,37.22761,-121.98071,South San Francisco,California,95030,Contemporary,$$$$,https://guide.michelin.com/us/en/california/so...
2,Benu,2019,37.78521,-122.39876,San Francisco,California,94105,Asian,$$$$,https://guide.michelin.com/us/en/california/sa...
3,Quince,2019,37.79762,-122.40337,San Francisco,California,94133,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...
4,Atelier Crenn,2019,37.79835,-122.43586,San Francisco,California,94123,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...


### Preprocessing

In [6]:
one_star_df['stars'] = pd.Series(0, index=one_star_df.index)
two_star_df['stars'] = pd.Series(1, index=two_star_df.index)
three_star_df['stars'] = pd.Series(2, index=three_star_df.index)

combined_df = pd.concat([one_star_df, two_star_df, three_star_df], axis=0).sample(frac=1.0).reset_index(drop=True)

In [7]:
combined_df

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
0,Zi Yat Heen,2019,22.145418,113.561520,Macau,Macau,NaN,Cantonese,$$,https://guide.michelin.com/mo/en/macau-region/...,0
1,Sushi Ryu,2019,25.055832,121.527240,Taipei,Taipei,110,Sushi,$$$$,https://guide.michelin.com/tw/en/taipei-region...,0
2,Madera,2019,37.420140,-122.211510,San Francisco,California,94025,Contemporary,$$$,https://guide.michelin.com/us/en/california/sa...,0
3,Summer Palace,2018,1.304385,103.825000,Singapore,Singapore,NaN,Cantonese,$,https://guide.michelin.com/sg/en/singapore-reg...,0
4,Ah Yat Harbour View (Tsim Sha Tsui),2019,22.296984,114.171970,Hong Kong,Hong Kong,NaN,Cantonese,$$,https://guide.michelin.com/hk/en/hong-kong-reg...,0
...,...,...,...,...,...,...,...,...,...,...,...
690,Aniar,2019,53.271120,-9.057078,Gaillimh/Galway,Ireland,NaN,Creative,NaN,https://guide.michelin.com/ie/en/galway/gailli...,0
691,Pramerl & the Wolf,2019,48.209450,16.371740,Wien,Austria,1090,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,0
692,Greenhouse,2019,53.340042,-6.258506,City Centre,Ireland,D2,Modern cuisine,NaN,https://guide.michelin.com/ie/en/dublin/city-c...,0
693,Rich Table,2019,37.774914,-122.422710,San Francisco,California,94102,Contemporary,$$$,https://guide.michelin.com/us/en/california/sa...,0


In [8]:
y = combined_df['stars'].copy()
X = combined_df.drop('stars', axis=1).copy()

In [9]:
y

0      0
1      0
2      0
3      0
4      0
      ..
690    0
691    0
692    0
693    0
694    0
Name: stars, Length: 695, dtype: int64

In [10]:
X

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Zi Yat Heen,2019,22.145418,113.561520,Macau,Macau,NaN,Cantonese,$$,https://guide.michelin.com/mo/en/macau-region/...
1,Sushi Ryu,2019,25.055832,121.527240,Taipei,Taipei,110,Sushi,$$$$,https://guide.michelin.com/tw/en/taipei-region...
2,Madera,2019,37.420140,-122.211510,San Francisco,California,94025,Contemporary,$$$,https://guide.michelin.com/us/en/california/sa...
3,Summer Palace,2018,1.304385,103.825000,Singapore,Singapore,NaN,Cantonese,$,https://guide.michelin.com/sg/en/singapore-reg...
4,Ah Yat Harbour View (Tsim Sha Tsui),2019,22.296984,114.171970,Hong Kong,Hong Kong,NaN,Cantonese,$$,https://guide.michelin.com/hk/en/hong-kong-reg...
...,...,...,...,...,...,...,...,...,...,...
690,Aniar,2019,53.271120,-9.057078,Gaillimh/Galway,Ireland,NaN,Creative,NaN,https://guide.michelin.com/ie/en/galway/gailli...
691,Pramerl & the Wolf,2019,48.209450,16.371740,Wien,Austria,1090,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
692,Greenhouse,2019,53.340042,-6.258506,City Centre,Ireland,D2,Modern cuisine,NaN,https://guide.michelin.com/ie/en/dublin/city-c...
693,Rich Table,2019,37.774914,-122.422710,San Francisco,California,94102,Contemporary,$$$,https://guide.michelin.com/us/en/california/sa...


In [11]:
X = X.drop(['name', 'zipCode', 'url'], axis=1)
X

,year,latitude,longitude,city,region,cuisine,price
0,2019,22.145418,113.561520,Macau,Macau,Cantonese,$$
1,2019,25.055832,121.527240,Taipei,Taipei,Sushi,$$$$
2,2019,37.420140,-122.211510,San Francisco,California,Contemporary,$$$
3,2018,1.304385,103.825000,Singapore,Singapore,Cantonese,$
4,2019,22.296984,114.171970,Hong Kong,Hong Kong,Cantonese,$$
...,...,...,...,...,...,...,...
690,2019,53.271120,-9.057078,Gaillimh/Galway,Ireland,Creative,NaN
691,2019,48.209450,16.371740,Wien,Austria,Creative,$$$$$
692,2019,53.340042,-6.258506,City Centre,Ireland,Modern cuisine,NaN
693,2019,37.774914,-122.422710,San Francisco,California,Contemporary,$$$


In [12]:
X.isna().sum()

year           0
latitude       0
longitude      0
city           2
region         0
cuisine        0
price        176
dtype: int64

In [13]:
X['price'].value_counts()

price
$$$$     197
$$$      143
$$        75
$$$$$     73
$         31
Name: count, dtype: int64

In [14]:
X['price'].mode().values[0]

'$$$$'

In [15]:
X['price'] = X['price'].fillna(X['price'].mode().values[0])
X.isna().sum()

year         0
latitude     0
longitude    0
city         2
region       0
cuisine      0
price        0
dtype: int64

In [16]:
# One hot encoding
{column : X[column].unique() for column in X.columns if X.dtypes[column]=='object'}

{'city': array(['Macau', 'Taipei', 'San Francisco', 'Singapore', 'Hong Kong',
        'Nottingham', 'Vejle', 'Seoul', 'Chicago', 'Göteborg',
        'Los Angeles', 'New York', 'Baslow', 'København', 'Stockholm',
        'Salzburg', 'Kleinwalsertal', 'Rio de Janeiro - 22271', 'Marlow',
        'South San Francisco', 'Oslo', 'Knowstone', 'Costa Mesa',
        'Washington, D.C.', 'Dubrovnik', 'Shinfield', 'Bangkok', 'Mayfair',
        'Corcaigh/Cork', 'Auchterarder', 'Birmingham', 'Aarhus',
        'Hampton in Arden', 'Harome', 'Saint Helier/Saint-Hélier',
        'Spitalfields', 'Helsingfors / Helsinki', 'Shoreditch',
        'Gaillimh/Galway', 'Egham', 'São Paulo - 01411', 'Waternish',
        'Athína', 'Budapest', 'Cartmel', 'São Paulo - 05415', 'Grasmere',
        'Warszawa', 'Marylebone', 'Clapham Common', 'Växjö', 'Malmesbury',
        'Järpen', 'Summerhouse', 'Chiswick', 'Clerkenwell',
        'Rio de Janeiro - 22021', 'Aird Mhór/Ardmore',
        'Baile Mhic Andáin/Thomastown', 'P

In [17]:
price_ordering = ['$', '$$', '$$$', '$$$$', '$$$$$']

X['price'] = X['price'].apply(lambda price: price_ordering.index(price))

X

,year,latitude,longitude,city,region,cuisine,price
0,2019,22.145418,113.561520,Macau,Macau,Cantonese,1
1,2019,25.055832,121.527240,Taipei,Taipei,Sushi,3
2,2019,37.420140,-122.211510,San Francisco,California,Contemporary,2
3,2018,1.304385,103.825000,Singapore,Singapore,Cantonese,0
4,2019,22.296984,114.171970,Hong Kong,Hong Kong,Cantonese,1
...,...,...,...,...,...,...,...
690,2019,53.271120,-9.057078,Gaillimh/Galway,Ireland,Creative,3
691,2019,48.209450,16.371740,Wien,Austria,Creative,4
692,2019,53.340042,-6.258506,City Centre,Ireland,Modern cuisine,3
693,2019,37.774914,-122.422710,San Francisco,California,Contemporary,2


In [18]:
# Getting rid of zip codes
X['city'] = X['city'].apply(lambda x: re.sub(r' - \d+$', '', x) if str(x) != 'nan' else x)
X['city'].unique()

array(['Macau', 'Taipei', 'San Francisco', 'Singapore', 'Hong Kong',
       'Nottingham', 'Vejle', 'Seoul', 'Chicago', 'Göteborg',
       'Los Angeles', 'New York', 'Baslow', 'København', 'Stockholm',
       'Salzburg', 'Kleinwalsertal', 'Rio de Janeiro', 'Marlow',
       'South San Francisco', 'Oslo', 'Knowstone', 'Costa Mesa',
       'Washington, D.C.', 'Dubrovnik', 'Shinfield', 'Bangkok', 'Mayfair',
       'Corcaigh/Cork', 'Auchterarder', 'Birmingham', 'Aarhus',
       'Hampton in Arden', 'Harome', 'Saint Helier/Saint-Hélier',
       'Spitalfields', 'Helsingfors / Helsinki', 'Shoreditch',
       'Gaillimh/Galway', 'Egham', 'São Paulo', 'Waternish', 'Athína',
       'Budapest', 'Cartmel', 'Grasmere', 'Warszawa', 'Marylebone',
       'Clapham Common', 'Växjö', 'Malmesbury', 'Järpen', 'Summerhouse',
       'Chiswick', 'Clerkenwell', 'Aird Mhór/Ardmore',
       'Baile Mhic Andáin/Thomastown', 'Praha', 'Finsbury', 'Victoria',
       'Bristol', 'Bath', 'Bloomsbury', 'Horsham', 'Soho', 'Ch

In [19]:
pd.get_dummies(X['cuisine'], prefix='C')

,C_American,C_Asian,C_Asian contemporary,C_Asian influences,C_Australian,C_Austrian,C_Barbecue,C_Californian,C_Cantonese,C_Cantonese Roast Meats,C_Chinese,C_Classic French,C_Classic cuisine,C_Contemporary,C_Creative,C_Creative British,C_Creative French,C_Danish,C_Dim Sum,C_European,C_European contemporary,C_Finnish,C_French,C_French contemporary,C_Fujian,C_Fusion,C_Gastropub,C_Hang Zhou,C_Hunanese and Sichuan,C_Indian,C_Innovative,C_International,C_Italian,C_Italian contemporary,C_Japanese,C_Japanese contemporary,C_Korean,C_Korean contemporary,C_Market cuisine,C_Meats and grills,C_Mediterranean,C_Mediterranean cuisine,C_Mexican,C_Modern British,C_Modern French,C_Modern cuisine,C_Moroccan,C_Noodles and congee,C_Peranakan,C_Regional cuisine,C_Scandinavian,C_Seafood,C_Shanghainese,C_Sichuan,C_Sichuan-Huai Yang,C_Southern Thai,C_Spanish,C_Steakhouse,C_Street Food,C_Sushi,C_Taiwanese,C_Taizhou,C_Temple cuisine,C_Teppanyaki,C_Thai,C_Thai Contemporary,C_Traditional British,C_Vegetarian,C_creative,C_modern
0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
691,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [20]:
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [21]:
X = onehot_encode(
    X,
    ['city', 'region', 'cuisine'],
    ['CI', 'RE', 'CU']
)

In [22]:
X

,year,latitude,longitude,price,CI_Aarhus,CI_Aird Mhór/Ardmore,CI_Anstruther,CI_Ascot,CI_Athína,CI_Auchterarder,CI_Aughton,CI_Bagshot,CI_Baile Mhic Andáin/Thomastown,CI_Ballydehob,CI_Baltimore,CI_Bangkok,CI_Baslow,CI_Bath,CI_Belfast,CI_Belgravia,CI_Bermondsey,CI_Biddenden,CI_Birkenhead,CI_Birmingham,CI_Blackrock,CI_Bloomsbury,CI_Bowness-on-Windermere,CI_Bray,CI_Bristol,CI_Budapest,CI_Burchett's Green,CI_Cambridge,CI_Cartmel,CI_Castle Combe,CI_Chagford,CI_Chelsea,CI_Cheltenham,CI_Chester,CI_Chew Magna,CI_Chicago,CI_Chiswick,CI_Cill Chainnigh/Kilkenny,CI_City Centre,CI_City of London,CI_Clapham Common,CI_Clerkenwell,CI_Colerne,CI_Corcaigh/Cork,CI_Costa Mesa,CI_Dalry,CI_Dorking,CI_Dubrovnik,CI_East Chisenbury,CI_Edinburgh,CI_Egham,CI_Fence,CI_Finsbury,CI_Fordwich,CI_Fredericia,CI_Fulham,CI_Gaillimh/Galway,CI_Grasmere,CI_Gravetye,CI_Great Milton,CI_Göteborg,CI_Hallwang,CI_Hammersmith,CI_Hampton in Arden,CI_Harome,CI_Helsingfors / Helsinki,CI_Henne,CI_Hong Kong,CI_Horsham,CI_Hunstanton,CI_Hyde Park,CI_Hørve,CI_Ilfracombe,CI_Järpen,CI_Kenilworth,CI_Kensington,CI_Kew,CI_Kleinwalsertal,CI_Knowstone,CI_København,CI_Langho,CI_Leeds,CI_Leith,CI_Leynar,CI_Lios Dúin Bhearna/Lisdoonvarna,CI_Little Dunmow,CI_Llanddewi Skirrid,CI_Llandrillo,CI_London,CI_Los Angeles,CI_Lovran,CI_Lympstone,CI_Macau,CI_Machynlleth,CI_Malmesbury,CI_Malmö,CI_Marlow,CI_Marylebone,CI_Mayfair,CI_Menai Bridge/Porthaethwy,CI_Monterey,CI_Montgomery/Trefaldwyn,CI_Morston,CI_Mountsorrel,CI_Murcott,CI_New York,CI_Newbury,CI_Newcastle upon Tyne,CI_North Kensington,CI_Nottingham,CI_Oldstead,CI_Oslo,CI_Oxford,CI_Padstow,CI_Pateley Bridge,CI_Peat Inn,CI_Pedersker,CI_Penarth,CI_Phuket,CI_Port Isaac,CI_Portscatho,CI_Praha,CI_Præstø,CI_Regent's Park,CI_Rio de Janeiro,CI_Ripley,CI_Rovinj,CI_Sacramento,CI_Saint Helier/Saint-Hélier,CI_Saint James's,CI_Salzburg,CI_San Diego,CI_San Francisco,CI_Seasalter,CI_Seoul,CI_Shinfield,CI_Shoreditch,CI_Singapore,CI_Skåne-Tranås,CI_Soho,CI_South Dalton,CI_South San Francisco,CI_Spitalfields,CI_Stavanger,CI_Stockholm,CI_Stratford-upon-Avon,CI_Summerhouse,CI_São Paulo,CI_Taipei,CI_Torquay,CI_Trondheim,CI_Upper Hambleton,CI_Vejle,CI_Victoria,CI_Växjö,CI_Wandsworth,CI_Warszawa,"CI_Washington, D.C.",CI_Waternish,CI_Westminster,CI_Whitebrook,CI_Wien,CI_Winchester,CI_Winteringham,CI_Zagreb,CI_Šibenik,RE_Austria,RE_California,RE_Chicago,RE_Croatia,RE_Czech Republic,RE_Denmark,RE_Finland,RE_Greece,RE_Hong Kong,RE_Hungary,RE_Ireland,RE_Macau,RE_New York City,RE_Norway,RE_Poland,RE_Rio de Janeiro,RE_Sao Paulo,RE_Singapore,RE_South Korea,RE_Sweden,RE_Taipei,RE_Thailand,RE_United Kingdom,RE_Washington DC,CU_American,CU_Asian,CU_Asian contemporary,CU_Asian influences,CU_Australian,CU_Austrian,CU_Barbecue,CU_Californian,CU_Cantonese,CU_Cantonese Roast Meats,CU_Chinese,CU_Classic French,CU_Classic cuisine,CU_Contemporary,CU_Creative,CU_Creative British,CU_Creative French,CU_Danish,CU_Dim Sum,CU_European,CU_European contemporary,CU_Finnish,CU_French,CU_French contemporary,CU_Fujian,CU_Fusion,CU_Gastropub,CU_Hang Zhou,CU_Hunanese and Sichuan,CU_Indian,CU_Innovative,CU_International,CU_Italian,CU_Italian contemporary,CU_Japanese,CU_Japanese contemporary,CU_Korean,CU_Korean contemporary,CU_Market cuisine,CU_Meats and grills,CU_Mediterranean,CU_Mediterranean cuisine,CU_Mexican,CU_Modern British,CU_Modern French,CU_Modern cuisine,CU_Moroccan,CU_Noodles and congee,CU_Peranakan,CU_Regional cuisine,CU_Scandinavian,CU_Seafood,CU_Shanghainese,CU_Sichuan,CU_Sichuan-Huai Yang,CU_Southern Thai,CU_Spanish,CU_Steakhouse,CU_Street Food,CU_Sushi,CU_Taiwanese,CU_Taizhou,CU_Temple cuisine,CU_Teppanyaki,CU_Thai,CU_Thai Contemporary,CU_Traditional British,CU_Vegetarian,CU_creative,CU_modern
0,2019,22.145418,113.561520,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [23]:
# Scale and Split X
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X

,year,latitude,longitude,price,CI_Aarhus,CI_Aird Mhór/Ardmore,CI_Anstruther,CI_Ascot,CI_Athína,CI_Auchterarder,CI_Aughton,CI_Bagshot,CI_Baile Mhic Andáin/Thomastown,CI_Ballydehob,CI_Baltimore,CI_Bangkok,CI_Baslow,CI_Bath,CI_Belfast,CI_Belgravia,CI_Bermondsey,CI_Biddenden,CI_Birkenhead,CI_Birmingham,CI_Blackrock,CI_Bloomsbury,CI_Bowness-on-Windermere,CI_Bray,CI_Bristol,CI_Budapest,CI_Burchett's Green,CI_Cambridge,CI_Cartmel,CI_Castle Combe,CI_Chagford,CI_Chelsea,CI_Cheltenham,CI_Chester,CI_Chew Magna,CI_Chicago,CI_Chiswick,CI_Cill Chainnigh/Kilkenny,CI_City Centre,CI_City of London,CI_Clapham Common,CI_Clerkenwell,CI_Colerne,CI_Corcaigh/Cork,CI_Costa Mesa,CI_Dalry,CI_Dorking,CI_Dubrovnik,CI_East Chisenbury,CI_Edinburgh,CI_Egham,CI_Fence,CI_Finsbury,CI_Fordwich,CI_Fredericia,CI_Fulham,CI_Gaillimh/Galway,CI_Grasmere,CI_Gravetye,CI_Great Milton,CI_Göteborg,CI_Hallwang,CI_Hammersmith,CI_Hampton in Arden,CI_Harome,CI_Helsingfors / Helsinki,CI_Henne,CI_Hong Kong,CI_Horsham,CI_Hunstanton,CI_Hyde Park,CI_Hørve,CI_Ilfracombe,CI_Järpen,CI_Kenilworth,CI_Kensington,CI_Kew,CI_Kleinwalsertal,CI_Knowstone,CI_København,CI_Langho,CI_Leeds,CI_Leith,CI_Leynar,CI_Lios Dúin Bhearna/Lisdoonvarna,CI_Little Dunmow,CI_Llanddewi Skirrid,CI_Llandrillo,CI_London,CI_Los Angeles,CI_Lovran,CI_Lympstone,CI_Macau,CI_Machynlleth,CI_Malmesbury,CI_Malmö,CI_Marlow,CI_Marylebone,CI_Mayfair,CI_Menai Bridge/Porthaethwy,CI_Monterey,CI_Montgomery/Trefaldwyn,CI_Morston,CI_Mountsorrel,CI_Murcott,CI_New York,CI_Newbury,CI_Newcastle upon Tyne,CI_North Kensington,CI_Nottingham,CI_Oldstead,CI_Oslo,CI_Oxford,CI_Padstow,CI_Pateley Bridge,CI_Peat Inn,CI_Pedersker,CI_Penarth,CI_Phuket,CI_Port Isaac,CI_Portscatho,CI_Praha,CI_Præstø,CI_Regent's Park,CI_Rio de Janeiro,CI_Ripley,CI_Rovinj,CI_Sacramento,CI_Saint Helier/Saint-Hélier,CI_Saint James's,CI_Salzburg,CI_San Diego,CI_San Francisco,CI_Seasalter,CI_Seoul,CI_Shinfield,CI_Shoreditch,CI_Singapore,CI_Skåne-Tranås,CI_Soho,CI_South Dalton,CI_South San Francisco,CI_Spitalfields,CI_Stavanger,CI_Stockholm,CI_Stratford-upon-Avon,CI_Summerhouse,CI_São Paulo,CI_Taipei,CI_Torquay,CI_Trondheim,CI_Upper Hambleton,CI_Vejle,CI_Victoria,CI_Växjö,CI_Wandsworth,CI_Warszawa,"CI_Washington, D.C.",CI_Waternish,CI_Westminster,CI_Whitebrook,CI_Wien,CI_Winchester,CI_Winteringham,CI_Zagreb,CI_Šibenik,RE_Austria,RE_California,RE_Chicago,RE_Croatia,RE_Czech Republic,RE_Denmark,RE_Finland,RE_Greece,RE_Hong Kong,RE_Hungary,RE_Ireland,RE_Macau,RE_New York City,RE_Norway,RE_Poland,RE_Rio de Janeiro,RE_Sao Paulo,RE_Singapore,RE_South Korea,RE_Sweden,RE_Taipei,RE_Thailand,RE_United Kingdom,RE_Washington DC,CU_American,CU_Asian,CU_Asian contemporary,CU_Asian influences,CU_Australian,CU_Austrian,CU_Barbecue,CU_Californian,CU_Cantonese,CU_Cantonese Roast Meats,CU_Chinese,CU_Classic French,CU_Classic cuisine,CU_Contemporary,CU_Creative,CU_Creative British,CU_Creative French,CU_Danish,CU_Dim Sum,CU_European,CU_European contemporary,CU_Finnish,CU_French,CU_French contemporary,CU_Fujian,CU_Fusion,CU_Gastropub,CU_Hang Zhou,CU_Hunanese and Sichuan,CU_Indian,CU_Innovative,CU_International,CU_Italian,CU_Italian contemporary,CU_Japanese,CU_Japanese contemporary,CU_Korean,CU_Korean contemporary,CU_Market cuisine,CU_Meats and grills,CU_Mediterranean,CU_Mediterranean cuisine,CU_Mexican,CU_Modern British,CU_Modern French,CU_Modern cuisine,CU_Moroccan,CU_Noodles and congee,CU_Peranakan,CU_Regional cuisine,CU_Scandinavian,CU_Seafood,CU_Shanghainese,CU_Sichuan,CU_Sichuan-Huai Yang,CU_Southern Thai,CU_Spanish,CU_Steakhouse,CU_Street Food,CU_Sushi,CU_Taiwanese,CU_Taizhou,CU_Temple cuisine,CU_Teppanyaki,CU_Thai,CU_Thai Contemporary,CU_Traditional British,CU_Vegetarian,CU_creative,CU_modern
0,0.243826,-0.839104,1.346851,-1.597431,-0.076084,-0.037959,-0.037959,-0.037959,-0.076084,-0.037959,-0.037959,-0.037959,-0.037959,-0.037959,-0.037959,-0.193167,-0.037959,-0.037959,-0.053722,-0.076084,-0.037959,-0.037959,-0.037959,-0.076084,-0.037959,-0.076084,-0.037959,-0.065843,-0.076084,-0.09331

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=40)

In [25]:
X_train

,year,latitude,longitude,price,CI_Aarhus,CI_Aird Mhór/Ardmore,CI_Anstruther,CI_Ascot,CI_Athína,CI_Auchterarder,CI_Aughton,CI_Bagshot,CI_Baile Mhic Andáin/Thomastown,CI_Ballydehob,CI_Baltimore,CI_Bangkok,CI_Baslow,CI_Bath,CI_Belfast,CI_Belgravia,CI_Bermondsey,CI_Biddenden,CI_Birkenhead,CI_Birmingham,CI_Blackrock,CI_Bloomsbury,CI_Bowness-on-Windermere,CI_Bray,CI_Bristol,CI_Budapest,CI_Burchett's Green,CI_Cambridge,CI_Cartmel,CI_Castle Combe,CI_Chagford,CI_Chelsea,CI_Cheltenham,CI_Chester,CI_Chew Magna,CI_Chicago,CI_Chiswick,CI_Cill Chainnigh/Kilkenny,CI_City Centre,CI_City of London,CI_Clapham Common,CI_Clerkenwell,CI_Colerne,CI_Corcaigh/Cork,CI_Costa Mesa,CI_Dalry,CI_Dorking,CI_Dubrovnik,CI_East Chisenbury,CI_Edinburgh,CI_Egham,CI_Fence,CI_Finsbury,CI_Fordwich,CI_Fredericia,CI_Fulham,CI_Gaillimh/Galway,CI_Grasmere,CI_Gravetye,CI_Great Milton,CI_Göteborg,CI_Hallwang,CI_Hammersmith,CI_Hampton in Arden,CI_Harome,CI_Helsingfors / Helsinki,CI_Henne,CI_Hong Kong,CI_Horsham,CI_Hunstanton,CI_Hyde Park,CI_Hørve,CI_Ilfracombe,CI_Järpen,CI_Kenilworth,CI_Kensington,CI_Kew,CI_Kleinwalsertal,CI_Knowstone,CI_København,CI_Langho,CI_Leeds,CI_Leith,CI_Leynar,CI_Lios Dúin Bhearna/Lisdoonvarna,CI_Little Dunmow,CI_Llanddewi Skirrid,CI_Llandrillo,CI_London,CI_Los Angeles,CI_Lovran,CI_Lympstone,CI_Macau,CI_Machynlleth,CI_Malmesbury,CI_Malmö,CI_Marlow,CI_Marylebone,CI_Mayfair,CI_Menai Bridge/Porthaethwy,CI_Monterey,CI_Montgomery/Trefaldwyn,CI_Morston,CI_Mountsorrel,CI_Murcott,CI_New York,CI_Newbury,CI_Newcastle upon Tyne,CI_North Kensington,CI_Nottingham,CI_Oldstead,CI_Oslo,CI_Oxford,CI_Padstow,CI_Pateley Bridge,CI_Peat Inn,CI_Pedersker,CI_Penarth,CI_Phuket,CI_Port Isaac,CI_Portscatho,CI_Praha,CI_Præstø,CI_Regent's Park,CI_Rio de Janeiro,CI_Ripley,CI_Rovinj,CI_Sacramento,CI_Saint Helier/Saint-Hélier,CI_Saint James's,CI_Salzburg,CI_San Diego,CI_San Francisco,CI_Seasalter,CI_Seoul,CI_Shinfield,CI_Shoreditch,CI_Singapore,CI_Skåne-Tranås,CI_Soho,CI_South Dalton,CI_South San Francisco,CI_Spitalfields,CI_Stavanger,CI_Stockholm,CI_Stratford-upon-Avon,CI_Summerhouse,CI_São Paulo,CI_Taipei,CI_Torquay,CI_Trondheim,CI_Upper Hambleton,CI_Vejle,CI_Victoria,CI_Växjö,CI_Wandsworth,CI_Warszawa,"CI_Washington, D.C.",CI_Waternish,CI_Westminster,CI_Whitebrook,CI_Wien,CI_Winchester,CI_Winteringham,CI_Zagreb,CI_Šibenik,RE_Austria,RE_California,RE_Chicago,RE_Croatia,RE_Czech Republic,RE_Denmark,RE_Finland,RE_Greece,RE_Hong Kong,RE_Hungary,RE_Ireland,RE_Macau,RE_New York City,RE_Norway,RE_Poland,RE_Rio de Janeiro,RE_Sao Paulo,RE_Singapore,RE_South Korea,RE_Sweden,RE_Taipei,RE_Thailand,RE_United Kingdom,RE_Washington DC,CU_American,CU_Asian,CU_Asian contemporary,CU_Asian influences,CU_Australian,CU_Austrian,CU_Barbecue,CU_Californian,CU_Cantonese,CU_Cantonese Roast Meats,CU_Chinese,CU_Classic French,CU_Classic cuisine,CU_Contemporary,CU_Creative,CU_Creative British,CU_Creative French,CU_Danish,CU_Dim Sum,CU_European,CU_European contemporary,CU_Finnish,CU_French,CU_French contemporary,CU_Fujian,CU_Fusion,CU_Gastropub,CU_Hang Zhou,CU_Hunanese and Sichuan,CU_Indian,CU_Innovative,CU_International,CU_Italian,CU_Italian contemporary,CU_Japanese,CU_Japanese contemporary,CU_Korean,CU_Korean contemporary,CU_Market cuisine,CU_Meats and grills,CU_Mediterranean,CU_Mediterranean cuisine,CU_Mexican,CU_Modern British,CU_Modern French,CU_Modern cuisine,CU_Moroccan,CU_Noodles and congee,CU_Peranakan,CU_Regional cuisine,CU_Scandinavian,CU_Seafood,CU_Shanghainese,CU_Sichuan,CU_Sichuan-Huai Yang,CU_Southern Thai,CU_Spanish,CU_Steakhouse,CU_Street Food,CU_Sushi,CU_Taiwanese,CU_Taizhou,CU_Temple cuisine,CU_Teppanyaki,CU_Thai,CU_Thai Contemporary,CU_Traditional British,CU_Vegetarian,CU_creative,CU_modern
193,0.243826,-1.307351,1.186234,0.464249,-0.076084,-0.037959,-0.037959,-0.037959,-0.076084,-0.037959,-0.037959,-0.037959,-0.037959,-0.037959,-0.037959,5.176872,-0.037959,-0.037959,-0.053722,-0.076084,-0.037959,-0.037959,-0.037959,-0.076084,-0.037959,-0.076084,-0.037959,-0.065843,-0.076084,-0.09331

In [26]:
y_train

193    1
163    2
463    0
218    0
651    0
      ..
440    1
165    0
7      2
219    0
326    0
Name: stars, Length: 486, dtype: int64

### Training

In [30]:
models = []
Cs = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

for i in range(len(Cs)):
    model = LogisticRegression(C=Cs[i])
    model.fit(X_train, y_train)
    models.append(model)

/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
models

[LogisticRegression(C=0.0001),
 LogisticRegression(C=0.001),
 LogisticRegression(C=0.01),
 LogisticRegression(C=0.1),
 LogisticRegression(),
 LogisticRegression(C=10.0),
 LogisticRegression(C=100.0)]

### Results

In [32]:
model_acc = [model.score(X_test, y_test) for model in models]

print(f" Model Accuracy (C={Cs[0]}):", model_acc[0])
print(f" Model Accuracy (C={Cs[1]}):", model_acc[1])
print(f" Model Accuracy (C={Cs[2]}):", model_acc[2])
print(f" Model Accuracy (C={Cs[3]}):", model_acc[3])
print(f" Model Accuracy (C={Cs[4]}):", model_acc[4])

 Model Accuracy (C=0.0001): 0.8133971291866029
 Model Accuracy (C=0.001): 0.8133971291866029
 Model Accuracy (C=0.01): 0.8038277511961722
 Model Accuracy (C=0.1): 0.7894736842105263
 Model Accuracy (C=1.0): 0.7607655502392344
